# ATTN: This script should be run AFTER spacy sentence splitting is completed.


# Use spacy to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_SENTENCIZED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
def get_word_num_and_frequency(row, text_col):

    with open(f'{data_dir}punctuations.txt', 'rb') as f:
        custom_punct_chars = pickle.load(f)
    row['Job Description num_words'] = len(str(row[text_col]).split())
    row['Job Description num_unique_words'] = len(set(str(row[text_col]).split()))
    row['Job Description num_chars'] = len(str(row[text_col]))
    row['Job Description num_chars_no_whitespact_and_punt'] = len(
        [
            c
            for c in str(row[text_col])
            if c not in custom_punct_chars and c not in string.punctuation and c != ' '
        ]
    )
    row['Job Description num_punctuations'] = len(
        [
            c
            for c in str(row[text_col])
            if c in custom_punct_chars and c in string.punctuation and c != ' '
        ]
    )

    return row


In [ ]:
def get_df_info(df, ivs_all=None):
    if ivs_all is None:
        ivs_all = [
            'Gender',
            'Gender_Num',
            'Gender_Female',
            'Gender_Mixed',
            'Gender_Male',
            'Age',
            'Age_Num',
            'Age_Older',
            'Age_Mixed',
            'Age_Younger',
        ]
    # Print Info
    print('\nDF INFO:\n')
    df.info()

    for iv in ivs_all:
        try:
            print('='*20)
            print(f'{iv}:')
            print('-'*20)
            if len(df[iv].value_counts()) > 5:
                print(f'{iv} Counts:\n{df[iv].value_counts()}')
                print('-'*20)
                print(f'{iv} Percentages:\n{df[iv].value_counts(normalize=True).mul(100).round(1).astype(float)}')
                print('-'*20)
            min_val = df[iv].min()
            max_val = df[iv].max()
            if min_val not in [0, 1]:
                print(f'Min {iv} value: {min_val}')
            if max_val not in [1, 3]:
                print(f'Max {iv} value: {max_val}')
            with contextlib.suppress(Exception):
                print('-'*20)
                print(f'{iv} Mean: {df[iv].mean().round(2).astype(float)}')
                print('-'*20)
                print(f'{iv} Standard Deviation: {df[iv].std().round(2).astype(float)}')
        except Exception:
            print(f'{iv} not available.')

    print('\n')


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_sentencized.pkl')


In [ ]:
# len=17599
df_jobs.info()

In [ ]:
# Job Ad info
get_df_info(df_jobs)


In [ ]:
# Explode df so that every row is one sentence
df_jobs = df_jobs.explode('Job Description spacy_sentencized', ignore_index=True)


In [ ]:
# 194820
len(df_jobs)


In [ ]:
df_jobs.info()


In [ ]:
df_jobs['Job Description spacy_sentencized_lower'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: job_sentence.strip().lower()
)


In [ ]:
df_jobs[['Job Description spacy_sentencized', 'Job Description spacy_sentencized_lower']].head()


In [ ]:
%%time
# Spacy tokenize
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_jobs['Job Description spacy_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        str(token.text.strip().lower())
        for token in nlp.tokenizer(job_sentence)
        if len(token) != 0
        and not token.is_space
        and not token.is_stop
        and not token.is_punct
        and not token.is_bracket
        and not token.like_email
        and not token.text in custom_punct_chars
    ]
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy.csv', index=False)


In [ ]:
df_jobs['Job Description spacy_sentencized_cleaned'] = df_jobs['Job Description spacy_tokenized'].str.join(' ')


In [ ]:
%%time
# Get sentence word frequencies
df_jobs = df_jobs.progress_apply(
    lambda row: get_word_num_and_frequency(
        row=row, text_col='Job Description spacy_sentencized'
    ), 
    axis='columns',
)


In [ ]:
df_jobs[
    [
        'Job Description spacy_sentencized',
        'Job Description num_words', 'Job Description num_unique_words',
        'Job Description num_chars', 'Job Description num_chars_no_whitespact_and_punt'
    ]
].head()

In [ ]:
df_jobs.columns


In [ ]:
# Job Sentence info
get_df_info(df_jobs)


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy.csv', index=False)


# Use NLTK to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy.pkl')


In [ ]:
df_jobs.info()


In [ ]:
%%time
# Tokenize with NLTK
stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

df_jobs['Job Description nltk_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentence: [
        str(token.strip().lower()) 
        for token in word_tokenize(job_sentence) 
        if len(token) != 0 
        and token != '...' 
        and not token.lower() in set(stopwords.words('english')) 
        and not token.lower() in list(string.punctuation) 
    ]
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.csv', index=False)


In [ ]:
df_jobs['Job Description nltk_tokenized'].head()


In [ ]:
df_jobs.info()


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.csv', index=False)


# Use gensim to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY_NLTK
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk.pkl')


In [ ]:
df_jobs.info()


In [32]:
%%time
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'
df_jobs['Job Description gensim_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: preprocess_string(re.sub(pattern, ' ', sentence.strip().lower()))
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.csv', index=False)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/IPython/cor │
│ e/magics/execution.py:1325 in time                                                               │
│                                                                                                  │
│   1322 │   │   else:                                                                             │
│   1323 │   │   │   st = clock2()                                                                 │
│   1324 │   │   │   try:                                                                          │
│ ❱ 1325 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1326 │   │   │   │   out=None                                                                  │
│   1327 │   │   │   │   # multi-line %%time case                                                  │
│   1328 │   │   │   │   if expr_val is not None:                                                  │
│ <timed exec>:7 in <module>                                                                       │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/pandas/core │
│ /generic.py:3064 in to_pickle                                                                    │
│                                                                                                  │
│    3061 │   │   """  # noqa: E501                                                                │
│    3062 │   │   from pandas.io.pickle import to_pickle                                           │
│    3063 │   │                                                                                    │
│ ❱  3064 │   │   to_pickle(                                                                       │
│    3065 │   │   │   self,                                                                        │
│    3066 │   │   │   path,                                                                        │
│    3067 │   │   │   compression=compression,                                                     │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/pandas/io/p │
│ ickle.py:112 in to_pickle                                                                        │
│                                                                                                  │
│   109 │   │   │   handles.handle.write(pickle.dumps(obj, protocol=protocol))                     │
│   110 │   │   else:                                                                              │
│   111 │   │   │   # letting pickle write directly to the buffer is more memory-efficient         │
│ ❱ 112 │   │   │   pickle.dump(obj, handles.handle, protocol=protocol)                            │
│   113                                                                                            │
│   114                                                                                            │
│   115 @doc(                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
df_jobs['Job Description gensim_tokenized'].head()


In [ ]:
df_jobs.info()


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.csv', index=False)


# Use BERT to tokenize sentences


### START HERE IF SOURCING FROM DF_JOBS_TOKENIZED_SPACY_NLTK_GENSIM
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [ ]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [ ]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim.pkl')


In [ ]:
df_jobs.info()


In [ ]:
%%time
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizerFast.from_pretrained(bert_model_name, strip_accents = True)
bert_model = BertForSequenceClassification.from_pretrained(bert_model_name).to(device)

df_jobs['Job Description bert_tokenized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: bert_tokenizer.tokenize(str(sentence))
)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_tokenized_spacy_nltk_gensim_bert.csv', index=False)


In [ ]:
df_jobs['Job Description bert_tokenized'].head()


In [ ]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_for_classification.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_for_classification.csv', index=False)
